In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd

ModuleNotFoundError: No module named 'bs4'

In [ ]:
def spiderSecHouse(addresslist, headers, houselist, cnt):
    for address in addresslist:
        print("开始抓取"+address+"区域的成交记录:")
        for i in range(1, 100):
            print("开始抓取" + address + "区域第"+str(i)+"页的成交记录:")
            if (i == 1):
                url = "https://sh.lianjia.com/ershoufang/" + address + "/ie2bt2sf1ba66ea120bp300ep530/"
            else:
                url = "https://sh.lianjia.com/ershoufang/" + address + "/pg" + str(i) + "ie2bt2sf1ba66ea120bp300ep530/"
            page = req.get(url, headers=headers)
            print(url + "页面状态码：{0}".format(page.status_code))
            soup = BeautifulSoup(page.text, "html.parser")

            listContent = soup.find_all(attrs={"class": "sellListContent"})
            if(len(listContent) == 0 ):
                break
            ullist = listContent[0].find_all(name='li')

            if(len(ullist) == 0):
                print("该页内容为空，跳过")
                break
            for ul in ullist:
                print("已经爬取" + str(cnt)+"条二手房在售信息")
                cnt += 1
                ifimg = ul.find_all("img")
                if (len(ifimg) == 2):
                    img_area = ul.find_all("img")[1].get('alt') #具体区域连
                else:
                    img_area = ""
                
                infolist = ul.find_all(attrs={"class": "info"})
                #网页地址
                href_connect = infolist[0].find_all(attrs={"class": ""})[0]
                lianjie = href_connect['href']
                # 标题，区域连
                
                alist = infolist[0].find_all(name='a')[0]
                split_titleinfo = alist.string.split(" ")

                if (len(split_titleinfo) == 3):
                    biaoti = split_titleinfo[0]
                    huxing = split_titleinfo[1]
                    area = split_titleinfo[2]
                else:
                    biaoti = split_titleinfo[0]
                    huxing = ""
                    area = img_area
                # 楼层、房屋建设时间
                #positionInfo = infolist[0].find(attrs={'class': "positionInfo"}).find('a').get_text()
                xiaoqu = infolist[0].find(attrs={'class': "positionInfo"}).find_all('a')[0].string
                positionInfo = infolist[0].find(attrs={'class': "positionInfo"}).find_all('a')[1].string
                #positionIcon = positionInfo[0].get_text().split(" ")
                #floor = positionIcon[0]
                #years = positionIcon[1]                
                
                # 房屋信息

                houseInfo = infolist[0].find_all(attrs={'class': "houseInfo"})
                split_houseinfo = houseInfo[0].get_text().split(" | ")
                if(len(split_houseinfo) == 7):
                    fangxing = split_houseinfo[0]
                    mianji = split_houseinfo[1]
                    chaoxiang = split_houseinfo[2]
                    zxtype = split_houseinfo[3]
                    floor = split_houseinfo[4]
                    year = split_houseinfo[5]
                    louxing = split_houseinfo[6]
                else:
                    fangxing = split_houseinfo[0]
                    mianji = split_houseinfo[1]
                    chaoxiang = split_houseinfo[2]
                    zxtype = split_houseinfo[3]
                    floor = split_houseinfo[4]
                    year = houseInfo[0].get_text()
                    louxing = ''
                # 关注人数，挂牌时间
                followInfo = infolist[0].find(attrs={'class':"followInfo"}).text.split("/")
                follow_nums = followInfo[0]
                guapai_time = followInfo[1]
                # 是否满五唯一
                five = infolist[0].find_all(attrs={'class':"five"})
                taxfree = infolist[0].find_all(attrs={'class':"taxfree"})
                #print(infolist[0])
                if(len(five) == 0 ):
                    if(len(taxfree) == 0 ):
                        manwu = ""
                    else:
                        manwu = taxfree[0].string
                else:
                    manwu = five[0].string
                # 成交总价
                totalPrice = infolist[0].find(attrs={'class':"totalPrice"}).find('span').string
                # 单价
                unitPrice = infolist[0].find(attrs={'class':"unitPrice"}).find('span').string
                # 挂牌价格、成交周期
                #dealCycleeInfo = infolist[0].find_all(attrs={'class': "dealCycleeInfo"})
                #dealCycleTxt = dealCycleeInfo[0].find_all(attrs={'class': "dealCycleTxt"})
                #dealCycleTxt_span = dealCycleTxt[0].find_all("span")
                #if (len(dealCycleTxt_span) == 2):
                #    listPrice = dealCycleTxt_span[0].get_text()
                #    dealCycle = dealCycleTxt_span[1].get_text()
                #elif (len(dealCycleTxt_span) == 1):
                #    if ("成交周期" in dealCycleTxt_span[0].get_text()):
                #        dealCycle = dealCycleTxt_span[0].get_text()
                #        listPrice = ""
                #    if ("挂牌" in dealCycleTxt_span[0].get_text()):
                #        listPrice = dealCycleTxt_span[0].get_text()
                #        dealCycle = ""

                houselist.append(
                    [address, biaoti,huxing,area,xiaoqu,positionInfo,fangxing,mianji,chaoxiang,zxtype,floor,year,louxing, \
                     follow_nums,guapai_time,manwu,totalPrice,unitPrice,lianjie])  



In [ ]:
if __name__ == "__main__":
    headers = {
        'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
                                   (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
    }
    # 浦东、杨浦、宝山
    addresslist = ["pudong", "minhang", "baoshan", "xuhui", "putuo", "yangpu", "changning", "songjiang", "jiading", \
                   "huangpu", "jingan", "hongkou", "qingpu"]
    # 最终爬取的成交信息
    houselist = []
    # 总记录数
    cnt = 0

    spiderSecHouse(addresslist, headers, houselist, cnt)

    #保存为csv
    columns = ["address","biaoti","huxing","area","xiaoqu","positionInfo","fangxing","mianji","chaoxiang","zxtype","floor", \
               "year","louxing","follow_nums","guapai_time","manwu","totalPrice","unitPrice","lianjie"] #

    df = pd.DataFrame(columns=columns, data=houselist)
    df.to_excel('lianjia_test1.xlsx')
    #df.to_csv("secHand_add.csv", mode='a', encoding="gbk")

    #secHand.csv 不带区域信息
    #secHand_address.csv 带区域信息 挂牌价格 成交周期有误
    #secHand_add.csv 带区域信息 挂牌价格 成交周期无误